In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
os.chdir('../')
path = os.getcwd()
print(path)

C:\home\program


In [3]:
data=pd.read_csv("scraping/COVID-19.csv")

C:\home\anaconda\envs\TF\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (1,2,3,4,16,17,23,24,25,31,32,42,43,53) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


データ参考
https://gis.jag-japan.com/covid19jp/

In [4]:
column=dict()
for prefecture in data["受診都道府県"]:
    if prefecture in column:
        column[prefecture]+=1
    else:
        column[prefecture]=1

日付の挿入
https://qiita.com/YuukiMiyoshi/items/42a890a95af6ab7a5348

In [5]:
date_list=data["確定日"]

In [8]:
#date_list = date_list.dropna()
#date_list = date_list.drop(date_list.index[[len(date_list)-1,len(date_list)-2,len(date_list)-3]])

In [9]:
# 使用するモジュールのインポート
from datetime import datetime as dt
from datetime import timedelta

# 日付条件の設定
strdt = dt.strptime(date_list[0], '%m/%d/%Y')  # 開始日
enddt = dt.strptime(date_list[len(date_list)-2], '%m/%d/%Y')  # 終了日

# 日付差の日数を算出（リストに最終日も含めたいので、＋１しています）
days_num = (enddt - strdt).days + 1  # （参考）括弧の部分はtimedelta型のオブジェクトになります

In [10]:
date = []
for i in range(days_num):
    d = strdt + timedelta(days=i)
    date.append(d.strftime("%m/%d/%Y"))

In [11]:
day=dict()
for word in date:
    day[word]=len(day)

In [12]:
covid19_jp=pd.DataFrame()
covid19_jp["date"]=date
for prefecture in column:
    positive=np.zeros(len(date))
    subdata=data[data["受診都道府県"]==prefecture]
    for word in subdata["確定日"]:
        if len(word)==10:
            pass
        else:
            word=dt.strptime(word, '%m/%d/%Y')
            word=word.strftime("%m/%d/%Y")
        positive[day[word]]+=1
    covid19_jp[prefecture]=positive

In [13]:
#positive=np.zeros(len(date))
#covid19_jp["岩手県"]=positive

In [14]:
covid19_jp.to_csv("scraping/covid19_pref_20200421.csv")

In [15]:
#column["岩手県"]=0

In [16]:
covid19_pref_accum=pd.DataFrame()
covid19_pref_accum["date"]=covid19_jp["date"]
[word]
for word in column:
    values=covid19_jp[word]
    results=np.zeros(len(values))
    value=0
    for i in range(len(values)):
        value+=values[i]
        results[i]=value
    covid19_pref_accum[word]=results

In [17]:
covid19_pref_accum.to_csv("scraping/covid19_pref_accum_20200421.csv")

In [18]:
region=dict()

In [19]:
region["Hokkaido"]=["北海道"]
region["Tohoku"]=["青森県","岩手県","宮城県","秋田県","山形県","福島県"]
region["Kanto"]=["茨城県","栃木県","群馬県","埼玉県","千葉県","東京都","神奈川県"]
region["Chubu"]=["新潟県","富山県","石川県","福井県","山梨県","長野県","岐阜県","静岡県","愛知県"]
region["Kinki"]=["三重県","滋賀県","京都府","大阪府","兵庫県","奈良県","和歌山県"]
region["Chugoku"]=["鳥取県","島根県","岡山県","広島県","山口県"]
region["Shikoku"]=["徳島県","香川県","愛媛県","高知県"]
region["Kyusyu"]=["福岡県","佐賀県","長崎県","熊本県","大分県","宮崎県","鹿児島県","沖縄県"]

In [20]:
covid19_region=pd.DataFrame()

In [21]:
covid19_region["date"]=covid19_jp["date"]

In [22]:
for word in region:
    values=np.zeros(len(covid19_jp))
    for word2 in region[word]:
        values=values+covid19_jp[word2]
    covid19_region[word]=values

In [23]:
covid19_region.to_csv("scraping/covid19_region_20200421.csv",index=False)

In [24]:
covid19_region

,date,Hokkaido,Tohoku,Kanto,Chubu,Kinki,Chugoku,Shikoku,Kyusyu
0,01/15/2020,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,01/16/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,01/17/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,01/18/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01/19/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
306,11/16/2020,0.0,0.0,180.0,0.0,0.0,0.0,0.0,0.0
307,11/17/2020,0.0,0.0,298.0,0.0,0.0,0.0,0.0,0.0
308,11/18/2020,0.0,0.0,493.0,0.0,0.0,0.0,0.0,0.0
309,11/19/2020,0.0,0.0,534.0,0.0,0.0,0.0,0.0,0.0


In [25]:
covid19_region_accumlation=pd.DataFrame()
covid19_region_accumlation["date"]=date

In [26]:
for word in region:
    values=covid19_region[word]
    results=np.zeros(len(values))
    value=0
    for i in range(len(values)):
        value+=values[i]
        results[i]=value
    covid19_region_accumlation[word]=results

In [27]:
covid19_region_accumlation

,date,Hokkaido,Tohoku,Kanto,Chubu,Kinki,Chugoku,Shikoku,Kyusyu
0,01/15/2020,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,01/16/2020,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,01/17/2020,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,01/18/2020,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,01/19/2020,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
306,11/16/2020,4444.0,1502.0,58043.0,9817.0,22961.0,1291.0,529.0,11051.0
307,11/17/2020,4444.0,1502.0,58341.0,9817.0,22961.0,1291.0,529.0,11051.0
308,11/18/2020,4444.0,1502.0,58834.0,9817.0,22961.0,1291.0,529.0,11051.0
309,11/19/2020,4444.0,1502.0,59368.0,9817.0,22961.0,1291.0,529.0,11051.0


In [28]:
covid19_region_accumlation.to_csv("scraping/covid19_region_accum_20200421.csv",index=False)